In [22]:
import numpy as np
import pandas as pd
import random

In [23]:
## Read all files
df_prior_research = pd.read_csv('../data/prior_research.csv')

array([    184,     734,   10067,     198,     639,    9272, 3718691,
            39,    2723, 7825200,   24901,   47184,      43,     236,
            16,       7,       2,      49,      24,      25,      41,
            63,     750,    4389,     278,     546,     819,    4700,
           291,     246,     759,    4299,     268,     346,     729,
          4000,     259,     550,    3218,     279,      61,      46,
            74,      55,      59,      54,      26,      45,       4,
           224,      14,      60,      15,     -46,     -27,    4486])

In [24]:
# Some useful function definitions

def w_star(n):
    w = 2*np.ones(n)/(3*n - 2)
    w[0] = n/(3*n - 2)
    return w

def w_cycle(n):
    w = np.ones(n)/n
    return w

def return_Ω(signals, truth, n_agents, n_samples):
    Ω = 0
    star_w = w_star(n_agents)
    cycle_w = w_cycle(n_agents)
    
    for i in range(n_samples):
        signal = np.asarray(random.choices(signals,k=n_agents))
        
        cycle_SAE = np.abs(cycle_w@signal - truth)
        star_SAE  = np.abs(star_w@signal - truth)
        
        if cycle_SAE > star_SAE:
            Ω+=1/n_samples
    return Ω

In [36]:
# Calculate Omega

n_samples = 10000
n_agents = 40

all_tasks = pd.DataFrame(columns=["task_id","study", "theta", "Omega_hat"])    

for idx,task_id in enumerate(df_prior_research['task_id'].unique()):
    df_temp = df_prior_research[df_prior_research['task_id']==task_id]
    
    signals = df_temp['pre_influence'].dropna().values
    theta = df_temp['pre_influence'].dropna().values[0]
    #print(theta)
    Omega_hat = return_Ω(signals, theta, n_agents, n_samples)
    
    all_tasks = all_tasks.append({
        "task_id": task_id,
        "study": df_temp['study'].dropna().values[0],
        "theta": theta,
        "Omega_hat": Omega_hat
    },ignore_index = True)

In [37]:
all_tasks.to_csv('../data/prior_tasks_with_omega.csv',index=False)

In [39]:
pd.set_option('display.max_rows', None)
all_tasks.sort_values("Omega_hat")

,task_id,study,theta,Omega_hat
20,gurcay2015_task_15_41.0,gurcay2015,41.0,0.2441
22,df_becker2017_750,becker2017,600.0,0.2684
38,df_becker2017_3218,becker2017,4100.0,0.2774
24,df_becker2017_278,becker2017,300.0,0.3137
36,df_becker2017_259,becker2017,264.0,0.3265
39,df_becker2017_279,becker2017,460.0,0.3286
31,df_becker2017_4299,becker2017,2476.0,0.3502
35,df_becker2017_4000,becker2017,5500.0,0.3534
23,df_becker2017_4389,becker2017,3000.0,0.3603
43,df_becker2017_55,becker2017,50.0,0.3619
